In [1]:
# import os
# os.environ["NUMBA_DISABLE_JIT"] = "1"

In [2]:
import sys, os
if 'google.colab' in sys.modules:
    %cd
    % rm -rf MPyDATA
    ! git clone --recurse-submodules -j8 https://github.com/Michaeldz36/MPyDATA.git
    %cd MPyDATA
    ! git checkout develop
    ! pip install -U $(cat requirements.txt | cut -d '=' -f 1)
else:
    sys.path.append(os.path.join(os.getcwd(), '../..'))

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from MPyDATA_examples.Olesik_et_al_2020.setup import Setup
from MPyDATA_examples.Olesik_et_al_2020.coordinates import x_id, x_p2, x_p3, x_log_of_pn
from MPyDATA.options import Options
from MPyDATA_examples.Olesik_et_al_2020.simulation import Simulation
from MPyDATA_examples.utils.show_plot import show_plot
from joblib import Parallel, parallel_backend, delayed
from MPyDATA_examples.Olesik_et_al_2020.physics.equilibrium_drop_growth import PdfEvolver
from MPyDATA.arakawa_c.discretisation import discretised_analytical_solution
from MPyDATA_examples.utils.error_norms import L2, Smolarkiewicz_Grabowski_1990_eq21
from MPyDATA_examples.Olesik_et_al_2020.convergence_plotter  import polar_plot

In [4]:
GCs = np.linspace(.05,.95, 4)
nrs = np.array([2**n for n in range(8, 15, 2)], dtype=int)

In [5]:
grid_layout = x_p2()
psi_coord = x_p2()
setup = Setup()

In [6]:
def analysis(setup, GC, opt):
    options = Options(**opt)
    simulation = Simulation(setup, grid_layout,  psi_coord, options, GC)
    simulation.step(simulation.out_steps[-1])
    t = simulation.out_steps[-1] * simulation.dt
    rh = simulation.rh
    pdf_t = PdfEvolver(setup.pdf, setup.drdt, t)
    def pdf_arg(r):
        return pdf_t(r* rh.units).magnitude
    analytical = discretised_analytical_solution(
                simulation.rh.magnitude,
                pdf_arg, midpoint_value = True
            ) * pdf_t(rh[0]).units
    numerical = simulation.n
    
    maximum_numeric = np.max(numerical)
    maximum_analytic = np.max(analytical)
    measure_height = (maximum_numeric / maximum_analytic).magnitude
    
    #TODO: proportion of rel dispersion
    
    error = np.log2(Smolarkiewicz_Grabowski_1990_eq21(numerical.magnitude, analytical.magnitude, t.magnitude))
    return setup.nr, GC, error, measure_height

In [10]:
opts = (
     {'n_iters': 1},
     {'n_iters': 2},
     {'n_iters': 2,'infinite_gauge':True},
     {'n_iters': 2, 'infinite_gauge': True, 'flux_corrected_transport': True},
     {'n_iters': 3},
     {'n_iters': 3, 'third_order_terms':True},
     {'n_iters': 3, 'third_order_terms': True, 'infinite_gauge': True, 'flux_corrected_transport': True}
)
def replace_names(opt):
    str_repl = [["'n_iters': 1","upwind"],
                ["'n_iters': 2","MPDATA 2 iterations"],
                ["'n_iters': 3","MPDATA 3 iterations"],
                ["'",""],
                [": True",""],
                ["_", " "],
                ["{",""],["}",""],[","," "]]                            
    for repl in str_repl:
        opt = str(opt).replace(repl[0], repl[1])
    return opt

In [ ]:
for opt in opts:
    with parallel_backend('threading', n_jobs=-2):
        results0 = Parallel(verbose=10)(
            delayed(analysis)(Setup(nr = nr, mixing_ratios_g_kg = [2,]), GC, opt)
            for nr in nrs
            for GC in GCs
        )
    results = tuple(tuple(i) for i in zip(*results0))
    plot_setup = np.array(results[0:2])
    measures = {'log$_2$(err)':results[2] , 'height difference':results[3]}
    opt = replace_names(opt)
    print(opt)
    for measure in measures.keys():
        polar_plot(nrs, GCs, measures[measure], name = measure)
        show_plot(filename = f'convergence_{measure}_{opt}.pdf')   #TODO: dont show plot name when savingfig (for masters)

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-2)]: Done  49 out of  56 | elapsed:  7.0min remaining:   59.6s


upwind


[Parallel(n_jobs=-2)]: Done  56 out of  56 | elapsed:  7.8min finished
C:\Users\Michael\PycharmProjects\MPyDATA\MPyDATA_examples\Olesik_et_al_2020\../..\MPyDATA_examples\Olesik_et_al_2020\convergence_plotter.py:30: UserWarning: The following kwargs were not used by contour: 'linewidth'
  plt.contour(X, Y, Z, levels, colors='black', linewidth=.3)
No handles with labels found to put in legend.


C:\Users\Michael\PycharmProjects\MPyDATA\MPyDATA_examples\Olesik_et_al_2020\../..\MPyDATA_examples\Olesik_et_al_2020\convergence_plotter.py:30: UserWarning: The following kwargs were not used by contour: 'linewidth'
  plt.contour(X, Y, Z, levels, colors='black', linewidth=.3)
No handles with labels found to put in legend.


[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-2)]: Done  49 out of  56 | elapsed:  9.1min remaining:  1.3min


...

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-2)]: Done  49 out of  56 | elapsed:  9.2min remaining:  1.3min


MPDATA 2 iterations  infinite gauge


[Parallel(n_jobs=-2)]: Done  56 out of  56 | elapsed: 10.4min finished
C:\Users\Michael\PycharmProjects\MPyDATA\MPyDATA_examples\Olesik_et_al_2020\../..\MPyDATA_examples\Olesik_et_al_2020\convergence_plotter.py:30: UserWarning: The following kwargs were not used by contour: 'linewidth'
  plt.contour(X, Y, Z, levels, colors='black', linewidth=.3)
No handles with labels found to put in legend.


C:\Users\Michael\PycharmProjects\MPyDATA\MPyDATA_examples\Olesik_et_al_2020\../..\MPyDATA_examples\Olesik_et_al_2020\convergence_plotter.py:30: UserWarning: The following kwargs were not used by contour: 'linewidth'
  plt.contour(X, Y, Z, levels, colors='black', linewidth=.3)
No handles with labels found to put in legend.


[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:  9.5min
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed: 12.7min
[Parallel(n_jobs=-2)]: Done  49 out of  56 | elapsed: 17.0min remaining:  2.4min
